# Warm up
Let's start with some warm up exercises to get you familiar with the database and do some SQL querying.
Make sure to have downloaded the NBA dataset form Kaggle as described in the README.

In [ ]:
import sqlite3 as sql
import pandas as pd
from pathlib import Path

## TODO: Use pathlib to get the path to the data directory. Path(__file__) won't work in a Jupyter notebook.
DATA_PATH = Path.cwd() / "data" / "nba.sqlite"

## Create a SQLite connection and run some SQL queries

In [ ]:
con = sql.connect(DATA_PATH)  # connect to the database
"query = SELECT * FROM game LIMIT 10;"  # write a query that selects the first 10 rows from the game table
top_10_game = pd.read_sql(query, con)  # read the query into a pandas dataframe
top_10_game.head()  # print the dataframe

In [ ]:
# let's look at the schema of the database
# TODO: sqlite_master is the name of the table you want to inspect: Replace TABLE_NAME with the name of the table you want to inspect
query = """
    SELECT *
    FROM sqlite_master
"""

pd.read_sql(query, con).sort_values('name')

In [ ]:
# let's see all the tables we have in the database
# TODO: Filter for tables only
query = """
    SELECT name
    FROM sqlite_master
    WHERE type='table'
"""
pd.read_sql(query, con)

In [ ]:
# let's have a look at the columns in the game table
query = """
    PRAGMA table_info(game)
"""
pd.read_sql(query, con)

In [ ]:
# let's find the earliest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date
    LIMIT 1
"""

print(pd.read_sql(query, con))

# let's find the latest recorded game
query = """
    SELECT game_date    
    FROM game
    ORDER BY game_date DESC
    LIMIT 1
"""

print(pd.read_sql(query, con))


In [ ]:
# TODO: Find four interesting facts about the data and write a query to find the answer

In [ ]:
query = """
SELECT first_name, 
       COUNT(first_name) * 100.0 / (SELECT COUNT(*) FROM officials) AS percentage_name
FROM officials
GROUP BY first_name
ORDER BY COUNT(first_name) DESC
LIMIT 3;
"""
finding1 = "Over 4% of all the officials are called Scott, with over 10% of officials having one of the 3 most popular names Scott, Tony or Mark"
print(pd.read_sql(query, con))
print(finding1)

In [ ]:
query = """
SELECT AVG(height_wo_shoes), AVG(height_w_shoes), AVG(height_w_shoes) - AVG(height_wo_shoes) AS avg_diff
FROM draft_combine_stats
;
"""
finding2 = "On average basketball players only gain an extra 1.19 inches when they put on their shoes - I thought it would be higher"
print(pd.read_sql(query, con))
print(finding2)

In [ ]:
query = """
    SELECT country, count(country) as country_total, MIN(from_year) as first_appearance
    FROM common_player_info
    GROUP BY country
    ORDER BY country_total  DESC
    LIMIT 5
"""
pd.read_sql(query, con)


In [ ]:
query = """
   SELECT 
    CASE
        WHEN country = 'USA' THEN 'United States'
        ELSE 'Other'
    END AS country_group,
    COUNT(*) AS total_players,
    ROUND(AVG(min_year)) AS avg_date_started_playing
FROM (
    SELECT 
        country,
        MIN(from_year) AS min_year
    FROM common_player_info
    GROUP BY person_id, country
    ) AS min_years_per_player
GROUP BY country_group;
"""

finding3 = "The vast majority of the NBA players are from the USA, however the sport seems to be becoming increasingly international as the average year that players from other countries started playing is 2010, compared to 1987 for US players, which is almost exactly the average of the length of time the NBA has been running"
print(pd.read_sql(query, con))
print(finding3)

In [ ]:
query = """
    SELECT COUNT(inactive_players.game_id) AS number_of_inactive_games, draft_combine_stats.player_id
    FROM inactive_players
    INNER JOIN draft_combine_stats ON inactive_players.player_id = draft_combine_stats.player_id
    GROUP BY inactive_players.game_id
    ORDER BY COUNT(inactive_players.game_id) DESC
"""

finding4 = "Newer players are more likely to be inactive"
print(pd.read_sql(query, con))
print(finding4)